# Preliminary CRISPR-Cas13b spacer analysis

The aim of this analysis is to get an initial feel on a small subset of Cas13b sequences and see if we see any genomic signals from the spacer sequences. This will also help to establish a analysis pipeline.

## Retrieve samples

Retrieve samples from Cas13_bacterial_db. Selection criteria are as follows:<br>
* Reported to carry the Cas13b gene
* Complete genome assembly
* Either sequenced by PacBio or Sanger (and presumabliy are high quality)


In [14]:
# query SQL database
import pandas as pd
from sqlalchemy import create_engine, text
import modules.database_utils as db_utils
from datetime import datetime

# connect to the database
creds = db_utils.get_credentials()['db_credentials']
database = 'cas13_bacterial_db'

pg_url = "postgresql+psycopg2://{0}:{1}@{2}:{3}/{4}".format(
                creds['user'], creds['password'], creds['host'], creds['port'], database)

engine = create_engine(pg_url, pool_pre_ping=True, echo=False)

query = """
    SELECT n.biosampleaccn,
        n.organism,
        c.cas13subtype,
        s.sequencingtechnology,
        n.ftppath_genbank
    FROM cas13_subtypes AS c
    JOIN ncbi_assembly_mono AS n ON c.organism = n.organism
    JOIN seq_platform AS s ON n.assemblyname = s.assemblyname
    WHERE c.cas13subtype = 'cas13b'
        AND n.assemblystatus = 'Complete Genome'
            AND n.exclfromrefseq = '[]'
            AND (s.sequencingtechnology LIKE '%PacBio%' OR s.sequencingtechnology LIKE '%Sanger%');
"""

# with engine.connect() as conn:
with engine.connect() as conn:
    df = pd.read_sql(text(query), conn.execution_options(stream_results=True))

timestamp = datetime.now().strftime('%d-%m-%Y-%H-%M-%S')

out_file = f"prelim/data/ncbi_assembly_{timestamp}.csv"

df.to_csv(out_file, index=False)

This resulted in a initial dataset of 105 samples. The next step is to pull down the fna, gbff and gff files from NCBI.

In [ ]:
import modules.genome_utils as genome_utils
from pathlib import Path
import os


prelim_data = db_utils.read_table_from_db(out_file)

out_file_dir = Path("prelim/data/genomes")

if not out_file_dir.exists():
    out_file_dir.mkdir(parents=True)

for index, row in prelim_data.iterrows():
    biosample_accession = row['biosampleaccn']
    ftp_path = row['ftppath_genbank']
    gff_file_path = genome_utils.get_gff_from_ncbi(ftp_path, biosample_accession, out_file_dir)
    if gff_file_path is None:
        pass
    elif os.path.exists(gff_file_path):
        genome_utils.get_fasta_from_ncbi(ftp_path, biosample_accession, out_file_dir)
        genome_utils.get_gbff_from_ncbi(ftp_path, biosample_accession, out_file_dir)

    biosampleaccn                                        organism  \
0    SAMN10691118         Apibacter raozihei (CFB group bacteria)   
1    SAMN08913567         Bergeyella cardium (CFB group bacteria)   
2    SAMN34037267         Bergeyella cardium (CFB group bacteria)   
3    SAMN32105916         Bergeyella cardium (CFB group bacteria)   
4    SAMN32105916         Bergeyella cardium (CFB group bacteria)   
..            ...                                             ...   
100  SAMN31358076  Elizabethkingia anophelis (CFB group bacteria)   
101  SAMN04875535  Elizabethkingia anophelis (CFB group bacteria)   
102  SAMN03996277  Elizabethkingia anophelis (CFB group bacteria)   
103  SAMN03996278  Elizabethkingia anophelis (CFB group bacteria)   
104  SAMN04875535  Elizabethkingia anophelis (CFB group bacteria)   

    cas13subtype      sequencingtechnology  \
0         cas13b                    PacBio   
1         cas13b                    PacBio   
2         cas13b                 

Extract only samples with Cas13b predicted. This information can be retrieved from the gbff and gff file.<br>

Look through files first, generate a master datasheet and only run crispridentify on those in the sheet.

In [ ]:
import pandas as pd
import modules.database_utils as db_utils
from modules.genome_utils import CasContig

df = db_utils.read_table_from_db("prelim/data/ncbi_assembly_30-06-2025-10-32-13.csv")
storage = Path("/spacers_db/genomes")

cas_type = "cas13b"

df_output = pd.DataFrame(columns=[
    'biosample_accession',
    'organism',
    'contig_id',
    'cas_gene',
    'locus_tag',
    'orientation',
    'start',
    'end'
])

rows = []

for index, row in df.iterrows():
    biosample_accession = row['biosampleaccn']
    print(biosample_accession)
    gbff_file = storage / f"{biosample_accession}.gbff"
    gff_file = storage / f"{biosample_accession}.gff"
    cls = CasContig.get_cas_info(gbff_file, gff_file)

    if cls is not None:
        # write to pandas dataframe
        df_row = {
            'biosample_accession': biosample_accession,
            'organism': row['organism'],
            'contig_id': cls.contig_id,
            'cas_gene': cls.cas_type,
            'locus_tag': cls.locus_tag,
            'orientation': cls.orientation,
            'start': cls.start,
            'end': cls.end
            }
        rows.append(df_row)
    else:
        print(f"No cas13b found for {biosample_accession} in {gbff_file} and {gff_file}")
        df_row = {
            'biosample_accession': biosample_accession,
            'organism': row['organism'],
            'contig_id': None,
            'cas_gene': None,
            'locus_tag': None,
            'orientation': None,
            'start': None,
            'end': None
        }
        rows.append(df_row)

df_output = pd.DataFrame(rows)

# Save the DataFrame to a CSV file
output_file = storage / "cas13b_contigs.csv"
df_output.to_csv(output_file, index=False)

This revealed that not all samples from the inital databaset thought to carry the Cas13b gene had the gene present. Instead, a total of 68 genomes out of 105 samples had the cas13b gene annotated.<br>

## CRISPR array prediction

Due to CRIPSRidentify being installed in a separate conda enviroment, have to run this separately from the other modules. 

In [ ]:
# run CRISPRidentify on the genomes that carries the cas13b gene
from modules.crispr_detecion import CrisprDetection
import pandas as pd


def read_table_from_db(file: Path) -> pd.DataFrame:
    """
    Reads a table from a database and returns it as a pandas DataFrame.

    args:
        file (Path): Path to the database file.
    """
    df = pd.read_csv(file, header=0)

    return df


storage = Path("/spacers_db/genomes")
df = read_table_from_db(f"{storage}/cas13b_contigs.csv")

for index, row in df.iterrows():
    if pd.isna(row['cas_gene']) or row['cas_gene'] == "":
        pass
    else:
        bioaccession = row['biosample_accession']
        fasta_file = storage / f"{bioaccession}.fna"
        # print(fasta_file)
        CrisprDetection.run_crispridentify(fasta_file, storage)

## Data upload

Once CRISPRidentify has completed running for all the samples, pull out results and upload to the cas13_bacterial_db. Results are written to two tables, cas13b_cripsrs and spacer_table. The cas13b_crisprs table contain information regarding all the predicted crispr array candidates and if they have passed the filter or not. 

Filter for array candidates the meet the following requirements:
* Located within 20kb of the Cas13b gene
* At least 4 spacers (this metric used by CRISPRCasFinder)
* Average spacer length around 30bp
* Average direct repeat length around 36bp

The spacer_table stores information and sequences of spacers and direct repeats from all the predicted crispr array candidates. 


In [ ]:
from pathlib import Path
import pandas as pd
from modules.analysis_utils import ArrayCandidate
from modules.genome_utils import CasContig
import modules.database_utils as db_utils



# read the cas13b contigs file
storage = Path("/spacers_db/genomes")
df = db_utils.read_table_from_db(f"{storage}/cas13b_contigs.csv")

for index, row in df.iterrows():
    # check if the cas_gene is not NaN or empty
    if pd.isna(row['cas_gene']) or row['cas_gene'] == "":
        pass
    else:
        # Run data extraction
        bioaccession = row['biosample_accession']
        gbff_file = storage / f"{bioaccession}.gbff"
        gff_file = storage / f"{bioaccession}.gff"
        result_folder = Path(f"/{storage}/{bioaccession}_crispridentify/{bioaccession}")

        candidates = ArrayCandidate.complie_crispr_arrays(result_folder, gbff_file, gff_file, storage)
        filter_candidates = ArrayCandidate.filter_candidates(candidates, avg_dr_len=36, avg_spacer_len=30)

        complete_candidates = []

        for c in filter_candidates:
            candidate = ArrayCandidate.get_spacer_dr_seq(c, result_folder)
            # print(candidate)
            if candidate is not None:
                complete_candidates.append(candidate)


        # Get the CasContig class instance for the given gbff and gff files
        # This will extract the contig_id and other relevant information
        cls = CasContig.get_cas_info(gbff_file, gff_file)


        array_df = db_utils.generate_array_table(complete_candidates, cls)
        spacer_df = db_utils.generate_spacer_table(complete_candidates)

        # Upload the dataframes to the SQL database
        db_utils.upload_arraytable_to_sql(array_df, "cas13_bacterial_db", "cas13b_crisprs")
        db_utils.upload_spacertable_to_sql(spacer_df, "cas13_bacterial_db", "spacer_table")


SAMN10691118
SAMN06754876
SAMN06289146
SAMEA104200672
SAMN05000968
SAMN06184581
SAMN13705423
SAMN47059527
SAMN04386100
SAMN04487986
SAMN28596590
SAMN18133624
SAMN25131779
SAMD00078003
SAMN04270979
SAMN11047481
SAMN02603919
SAMN39216401
SAMN41998628
SAMN43911957
SAMN40623457
SAMN45156075
SAMN45156076
SAMN45934160
SAMN10230162
SAMN09907746
SAMN19113859
SAMN03761967
SAMEA4063029
SAMN04394637
SAMN13111529
SAMN07840095
SAMN07836928
SAMN07840081
SAMN07840090
SAMN07840091
SAMN07840092
SAMN07840094
SAMN07840097
SAMN07831761
SAMN07836904
SAMN07836934
SAMN07315161
SAMN07315160
SAMN07315163
SAMN18805464
SAMN18805165
SAMN18753385
SAMN35301467
SAMN35563277
SAMD00060990
SAMN03366764
SAMN03372093
SAMN03653671
SAMN25263445
SAMN02603988
SAMD00034934
SAMN04529095
SAMN04529094
SAMN07955957
SAMN07956007
SAMN07955952
SAMN07956008
SAMN07956010
SAMN07955950
SAMN07980911
SAMN07980959
SAMN07955960


## Sequence analysis

# In progress

Retrieved samples from the Cas13 bacterial database, searching through the ncbi_sra and ncbi_assembly_mono tables for the organism *Bacteroidales bacterium*, *Bergeyella zoohelcum*, *Segatella buccae* and *Prevotella pectinovora*, all reported to have the CRIPSR-Cas13b system.<br>

Grab the data from the ncbi_sra.csv file and see how many samples are within allthebacteria database so we can use the assembly from there, avoid having to perform de novo assembly.

In [ ]:
from pathlib import Path
import modules.database_utils as db_utils
from modules.database_utils import Allthebacteria


ncbi_sra_file = Path("/home/unimelb.edu.au/rbengtsson/work/spacer_analysis/prelim/data/ncbi_sra.csv")

sra_df, atb_merged_df = Allthebacteria.check_SRA_df(ncbi_sra_file)




A total of 92 samples from the 696 SRA samples are in the Allthebacteria database. We then need to inspect the ncbi_assembly.csv file and cross reference with samples within Allthebacteria datbase to ensure we don't have any redundant samples present across both sets.

In [2]:
# check for samples present in the assembly file and Allthebacteria file
assembly_df = db_utils.read_table_from_db("prelim/data/ncbi_assembly.csv")
assembly_df['in_allthebacteria'] = assembly_df['biosampleaccn'].isin(atb_merged_df['sample_accession'])
print(f"A total of {assembly_df['in_allthebacteria'].sum()} samples from the {len(assembly_df)} NCBI assembly samples are in Allthebacteria database.")

A total of 5 samples from the 44 NCBI assembly samples are in Allthebacteria database.


Drop the 5 samples from the NCBI assembly dataset and use the ones in Allthebacteria dataset

In [7]:
assembly_df = assembly_df[assembly_df['in_allthebacteria'] == False].reset_index(drop=True)

In [8]:
print(f"Perform analysis on {len(assembly_df)} samples from NCBI assembly and {len(atb_merged_df)} samples from Allthebacteria database.")

Perform analysis on 39 samples from NCBI assembly and 91 samples from Allthebacteria database.


In [19]:
import pandas as pd
import os

# Select relevant columns from the assembly dataframe
columns_to_keep = ['biosampleaccn', 'organism', 'speciesname',
                   'contign50', 'sequencingtechnology', 'ftppath_genbank', 'ftppath_assembly_rpt']
assembly_subset_df = assembly_df[columns_to_keep]
assembly_subset_df['database'] = 'ncbi assembly'

atb_merged_df['database'] = 'allthebacteria'

# Change column names to match Allthebacteria
atb_merged_df.rename(columns={
    'sample_accession': 'biosampleaccn',
    'instrument_model': 'sequencingtechnology',
    'n50': 'contign50'
}, inplace=True)


# Concatenate the two dataframes
combined_df = pd.merge(assembly_subset_df, atb_merged_df, on=['biosampleaccn', 'organism','sequencingtechnology', 'contign50', 'database'], how='outer')

# Save the combined dataframe to a CSV file
out_file = os.path.join('prelim', 'data', 'prelim_dataset_final.csv')
combined_df.to_csv(out_file, index=False)


/tmp/ipykernel_4024695/4152738273.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assembly_subset_df['database'] = 'ncbi assembly'


Once we have the dataset, we now need to retrieve the fasta file. For samples coming from NCBI assembly database, we can directly fetch the fasta and gff files from the ftp provided.<br>

Allthebacteria dataset only have assemblies avaliable. Going to continue with the 39 samples from NCBI assembly database for now, as genome annotation is avaliable for those samples. 

In [ ]:
import modules.genome_utils as genome_utils

prelim_data = db_utils.read_table_from_db("prelim/data/prelim_dataset_final.csv")

# Get the fasta files for the samples in the prelim_data
ncbi_assembly_df = prelim_data[prelim_data['database'] == 'ncbi assembly']

out_file_dir = Path("prelim/data/genomes")

if not out_file_dir.exists():
    out_file_dir.mkdir(parents=True)

# for index, row in ncbi_assembly_df.iterrows():
#     biosample_accession = row['biosampleaccn']
#     ftp_path = row['ftppath_genbank']
#     gff_file_path = genome_utils.get_gff_from_ncbi(ftp_path, biosample_accession, out_file_dir)
#     if gff_file_path is None:
#         pass
#     elif os.path.exists(gff_file_path):
#         genome_utils.get_fasta_from_ncbi(ftp_path, biosample_accession, out_file_dir)

Majority of the samples were suppressed from NCBI and we are left with only 14 samples for the preliminary analysis.<br>

Have to grab assemblies from allthebacteria.

In [ ]:
# # Get the fasta files for the samples in the prelim_data
# atb_assembly_df = prelim_data[prelim_data['database'] == 'allthebacteria']

# for index, row in atb_assembly_df.iterrows():
#     biosample_accession = row['biosampleaccn']
#     genome_utils.get_fasta_from_allthebacteria(biosample_accession, out_file_dir)

## Genome annotation

Once the assemblies have been retrieved, the next step is to perform genome annotation using PGAP.

## Run CasFinder

Search for cas13b gene in samples
